In [76]:
import requests_cache
from datetime import timedelta
import json
from src.info import get_address_info
from src.ens import address_from_ens, ens_from_address, other_ens_owned_by, domain_events
from src.transactions import get_transactions
from src.tokens import get_token_transfers
from src.time_series import get_time_series
from src.graph import count_transactions_by_address
from typing import List, Tuple, Dict

session = requests_cache.CachedSession('./cache',
                                       expire_after=timedelta(days=3))

# INPUT = "0x71B5b8Ec4D724849E76954A1475930dcE9c4B994"
INPUT = "kartik.eth"
INPUT = "poap.eth"
input = INPUT

if input.endswith(".eth"):
    input = address_from_ens(session, input)
ens = ens_from_address(session, input)
other_ens_owned = other_ens_owned_by(session, input)
addresses_with_domain_events = []
for e in ens:
    addresses_with_domain_events.extend(domain_events(session, e))
transactions = get_transactions(session, input)
token_transfers = get_token_transfers(session, input)

In [77]:
other_ens_owned

[{'name': 'poap.xyz', 'address': None},
 {'name': 'checkout.poap.eth',
  'address': '0xD648b4BE702Ff3651FBDB7be18c3548986A7D058'},
 {'name': 'sales.poap.eth',
  'address': '0xe6DBF43F5927969Fd99E3d193C3fBfB88951A2B9'},
 {'name': 'vault.poap.eth',
  'address': '0x90A9d3661B1963BC33ccF4D55d663dF5856C1126'},
 {'name': 'booth.poap.eth', 'address': None},
 {'name': 'ecosystem.poap.eth',
  'address': '0xdDd41BCd80FEb8B02e46baDA482Eb6AFD433aff7'},
 {'name': 'community.poap.eth', 'address': None},
 {'name': 'engineering.poap.eth',
  'address': '0x72b1784aAA95E2451D2FF567181Ec11961aC3eA1'},
 {'name': 'ops.poap.eth',
  'address': '0xF99AC673d70D3758E3291C37463D0D26e0E055a4'},
 {'name': 'multisig.poap.eth',
  'address': '0x059b3e91ea520436AE0685c931681c489d8375dE'},
 {'name': 'heidi.eth', 'address': None}]

In [78]:
nodes = [
    {"id": 0, "label": "\n".join(ens), "group": 1, "size": 20, "address": input},
]
edges = []
nodes

[{'id': 0,
  'label': 'poaplabs.eth\npoap.9dcc.eth\npoap.mirror.xyz\nmbag.eth\npoapcollection.eth\npoap.sismo.eth\njuliuspreite.eth\nlavarello.eth\npoap.eth\npatricio.eth',
  'group': 1,
  'size': 20,
  'address': '0xf6B6F07862A02C85628B3A9688beae07fEA9C863'}]

In [79]:
def add_address_to_nodes(address: str, strength: int, label: str = None):
    if address == input:
        return
    node = next((a for a in nodes if a["address"] == address), None)
    if node is None:
        nodes.append({
            "id": len(nodes),
            "label": label if label else address,
            "group": 2,
            "size": 10,
            "address": address
        })
        node = nodes[-1]
    
    edge = next((e for e in edges if e["to"] == node["id"]), None)
    if edge is None:
        edges.append({
            "from": 0,
            "to": node["id"],
            "strength": strength
        })
    else:
        edges[edges.index(edge)]["strength"] += strength


In [80]:
addresses_with_domain_events = set(addresses_with_domain_events)
if input in addresses_with_domain_events:
    addresses_with_domain_events.remove(input)
for address in addresses_with_domain_events:
    add_address_to_nodes(address, 1)

In [81]:
len(nodes)

3

In [82]:
for e in other_ens_owned:
    if e["address"]:
        add_address_to_nodes(e["address"], 1, e["name"])

In [83]:
def count_transactions_by_address(
        transactions: List[dict]) -> List[Tuple[str, int, int]]:
    transactions = [
        tx for tx in transactions
        if not tx["contract"]
    ]
    address_counts: Dict[str, Tuple[int, int]] = {}

    for tx in transactions:
        if tx['from'] not in address_counts:
            address_counts[tx['from']] = (0, 0)
        if tx['to'] not in address_counts:
            address_counts[tx['to']] = (0, 0)

        from_in, from_out = address_counts[tx['from']]
        to_in, to_out = address_counts[tx['to']]

        address_counts[tx['from']] = (from_in, from_out + 1)
        address_counts[tx['to']] = (to_in + 1, to_out)

    result = [(addr, in_count, out_count)
              for addr, (in_count, out_count) in address_counts.items()]
    return sorted(result, key=lambda x: x[1] + x[2], reverse=True)

In [84]:
top_interactions = count_transactions_by_address(transactions + token_transfers)[:10]
max_amount = (top_interactions[1][1] + top_interactions[1][2]) / 2
for i in top_interactions:
    add_address_to_nodes(i[0], (i[1] + i[2]) / max_amount)

In [85]:
edges

[{'from': 0, 'to': 1, 'strength': 1},
 {'from': 0, 'to': 2, 'strength': 1},
 {'from': 0, 'to': 3, 'strength': 1},
 {'from': 0, 'to': 4, 'strength': 1},
 {'from': 0, 'to': 5, 'strength': 1},
 {'from': 0, 'to': 6, 'strength': 1.8},
 {'from': 0, 'to': 7, 'strength': 1},
 {'from': 0, 'to': 8, 'strength': 1},
 {'from': 0, 'to': 9, 'strength': 1},
 {'from': 0, 'to': 10, 'strength': 2.0},
 {'from': 0, 'to': 11, 'strength': 1.0},
 {'from': 0, 'to': 12, 'strength': 1.0},
 {'from': 0, 'to': 13, 'strength': 0.8},
 {'from': 0, 'to': 14, 'strength': 0.6},
 {'from': 0, 'to': 15, 'strength': 0.6},
 {'from': 0, 'to': 16, 'strength': 0.6},
 {'from': 0, 'to': 17, 'strength': 0.4}]

In [87]:
for n in nodes:
    if n["label"].startswith("0x") and len(n["label"]) == 42:
        ens = ens_from_address(session, n["label"])
        if len(ens) > 0:
            nodes[nodes.index(n)]["label"] = ens[0]            

{'id': 1, 'label': '0x0904Dac3347eA47d208F3Fd67402D039a3b99859', 'group': 2, 'size': 10, 'address': '0x0904Dac3347eA47d208F3Fd67402D039a3b99859'}
{'id': 2, 'label': '0x4430eDFBB4777B3F8E5b951803657703039D688b', 'group': 2, 'size': 10, 'address': '0x4430eDFBB4777B3F8E5b951803657703039D688b'}
{'id': 10, 'label': '0x4CCC2ACdd43811Ce928cef7371Fc95daF7418631', 'group': 2, 'size': 10, 'address': '0x4CCC2ACdd43811Ce928cef7371Fc95daF7418631'}
{'id': 11, 'label': '0xbAED477c4A8Cdb6079E4A64950ac60941b9A301f', 'group': 2, 'size': 10, 'address': '0xbAED477c4A8Cdb6079E4A64950ac60941b9A301f'}
{'id': 12, 'label': '0x7DBF6820D32cFBd5D656bf9BFf0deF229B37cF0E', 'group': 2, 'size': 10, 'address': '0x7DBF6820D32cFBd5D656bf9BFf0deF229B37cF0E'}
{'id': 13, 'label': '0x20D08E58952bb70ff0EC1271085cA969C14BE2a3', 'group': 2, 'size': 10, 'address': '0x20D08E58952bb70ff0EC1271085cA969C14BE2a3'}
{'id': 14, 'label': '0xA1DD0b787E5ee1fe2C50cEcb5560b543527F4053', 'group': 2, 'size': 10, 'address': '0xA1DD0b787E5ee1fe

In [ ]:
nodes